<a href="https://colab.research.google.com/github/tech09876/pytorch_covid19_x-ray_detection/blob/main/diabetes_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors
import math

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import torch 
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm_notebook
import torch.nn.functional as F
import torch.nn as nn
from sklearn.metrics import accuracy_score 
import torch.optim 
import pickle

In [ ]:
torch.manual_seed(20)

In [ ]:
diabetes_dataset = pd.read_csv('/content/diabetes.csv') 

In [ ]:
diabetes_dataset.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
diabetes_dataset.shape

(768, 9)

In [ ]:
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
diabetes_dataset['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [ ]:
diabetes_dataset.groupby('Outcome').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [ ]:
# separating the data and labels
X = diabetes_dataset.drop(columns = 'Outcome', axis=1).values
Y = diabetes_dataset['Outcome'].values
print(X)
print(Y)

[[  6.    148.     72.    ...  33.6     0.627  50.   ]
 [  1.     85.     66.    ...  26.6     0.351  31.   ]
 [  8.    183.     64.    ...  23.3     0.672  32.   ]
 ...
 [  5.    121.     72.    ...  26.2     0.245  30.   ]
 [  1.    126.     60.    ...  30.1     0.349  47.   ]
 [  1.     93.     70.    ...  30.4     0.315  23.   ]]
[1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0
 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1
 1 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 0 0 0
 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1
 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0
 1 0 1 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 1 0 0 0 1 1 1 0 0
 1 0 1 0 1 1 0 1 0 0 1 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 0

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=0)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(614, 8) (154, 8) (614,) (154,)


In [ ]:
###Creating Tensors###
X_train=torch.FloatTensor(X_train)
X_test=torch.FloatTensor(X_test)
Y_train=torch.LongTensor(Y_train)
Y_test=torch.LongTensor(Y_test)

In [ ]:
class FirstNetwork_v1(nn.Module):
    def __init__(self,input_features=8,hidden1=100,hidden2=50,out_features=2):
        super().__init__()
        self.f_connected1=nn.Linear(input_features,hidden1)
        self.f_connected2=nn.Linear(hidden1,hidden2)
        self.out=nn.Linear(hidden2,out_features)
    def forward(self,x):
        x=F.relu(self.f_connected1(x))
        x=F.relu(self.f_connected2(x))
        x=self.out(x)
        return x
      

In [ ]:
def fit_v1(x,y, model, opt, loss_fn, epochs=100000):
  for epoch in tqdm_notebook(range(epochs),total=epochs, unit='epoch'):
    loss= loss_fn(model(x),y)

    loss.backward()
    opt.step()
    opt.zero_grad()

  return loss.item()

In [ ]:
%%time
fn = FirstNetwork_v1()
loss_fn = F.cross_entropy
opt = torch.optim.SGD(fn.parameters(),lr=0.1)
fit_v1(X_train, Y_train, fn, opt, loss_fn)



  0%|          | 0/100000 [00:00<?, ?epoch/s]

CPU times: user 2min 46s, sys: 1.5 s, total: 2min 47s
Wall time: 2min 52s


0.6458358764648438

In [ ]:
prediction=[]
with torch.no_grad():
  for i,data in enumerate(X_test):
    y_pred= fn(data)
    prediction.append(y_pred.argmax().item())
    print(y_pred.argmax().item())

In [ ]:
score = accuracy_score(Y_test, prediction)
score

0.6493506493506493

In [ ]:
lst1=[5,166,72,19,175,25.8,0.587,51]

In [ ]:
new_data=torch.tensor(lst1)

In [ ]:
with torch.no_grad():
    print(fn(new_data))
    print(fn(new_data).argmax().item())

tensor([ 0.2636, -0.3594])
0


In [ ]:
torch.save(fn,'diabetes.pt')

In [ ]:
filename = 'diabetes_model.sav'
pickle.dump(fn, open(filename, 'wb'))